In [1]:
%pip install -U transformers 
%pip install -U datasets 
%pip install -U accelerate 
%pip install -U peft 
%pip install -U trl 
%pip install -U bitsandbytes 
%pip install -U wandb
%pip install langchain
%pip install langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 69.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.0
    Uninstalling transformers-4.44.0:
      Successfully uninstalled transformers-4.44.0
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 4.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 6.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 11.6 MB/s eta 0:00:00
Note: you may

In [2]:
import os
import requests
from transformers import pipeline
from pydantic import BaseModel
from langchain import PromptTemplate
from langchain.tools import tool
from langgraph.graph import StateGraph
import torch
import time
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch, wandb
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format

In [3]:
name = "Sneha"

In [4]:
with open("/kaggle/input/sneha-parse/Sneha_Parse.txt","r") as f:
  raw_content=f.read()
print(raw_content[:1000])
i=0

while(raw_content[i]=='\n'):
  i+=1
j=len(raw_content)-1

while(raw_content[i]=='\n'):
  j-=1
raw_content=raw_content[i:j]
raw_content[:1000]


Scenario 1 (Group 1):
Main Question: I'm interviewing Sneha. Hi, Sneha.
Answer: Hello, Nidi. How are you? I'm good.
Follow-up Question: Very good. Can you tell me about your background including your age group, profession, and education level?
Answer: I'm 21, and I'm pursuing pursuing BTEC CSA.
Follow-up Question: How would you describe your personality type?
Answer: My personality type? Someone who loves love. Someone who loves human. Someone who loves to take a walk around. Okay.
Follow-up Question: How often do you experience stress or anxiety, and what methods do you use to cope with it?
Answer: Oh, I experience anxiety a lot lately, and it's been hard to cope with it. But I'm trying to, you know, get my way out of it. So I'm trying new things. I try to wake up morning in the morning, and I go outside, take a walk. I just speak with myself, listen to the song that gives me hope, that gives me vision for my future and the things that I wanna do in life. That's how I do.
Follow-up Q

"Scenario 1 (Group 1):\nMain Question: I'm interviewing Sneha. Hi, Sneha.\nAnswer: Hello, Nidi. How are you? I'm good.\nFollow-up Question: Very good. Can you tell me about your background including your age group, profession, and education level?\nAnswer: I'm 21, and I'm pursuing pursuing BTEC CSA.\nFollow-up Question: How would you describe your personality type?\nAnswer: My personality type? Someone who loves love. Someone who loves human. Someone who loves to take a walk around. Okay.\nFollow-up Question: How often do you experience stress or anxiety, and what methods do you use to cope with it?\nAnswer: Oh, I experience anxiety a lot lately, and it's been hard to cope with it. But I'm trying to, you know, get my way out of it. So I'm trying new things. I try to wake up morning in the morning, and I go outside, take a walk. I just speak with myself, listen to the song that gives me hope, that gives me vision for my future and the things that I wanna do in life. That's how I do.\nFo

In [5]:
questions=raw_content.split("\n\n")
print(questions[:5])

["Scenario 1 (Group 1):\nMain Question: I'm interviewing Sneha. Hi, Sneha.\nAnswer: Hello, Nidi. How are you? I'm good.\nFollow-up Question: Very good. Can you tell me about your background including your age group, profession, and education level?\nAnswer: I'm 21, and I'm pursuing pursuing BTEC CSA.\nFollow-up Question: How would you describe your personality type?\nAnswer: My personality type? Someone who loves love. Someone who loves human. Someone who loves to take a walk around. Okay.\nFollow-up Question: How often do you experience stress or anxiety, and what methods do you use to cope with it?\nAnswer: Oh, I experience anxiety a lot lately, and it's been hard to cope with it. But I'm trying to, you know, get my way out of it. So I'm trying new things. I try to wake up morning in the morning, and I go outside, take a walk. I just speak with myself, listen to the song that gives me hope, that gives me vision for my future and the things that I wanna do in life. That's how I do.\nF

In [6]:
questions_processed=[]
for q in questions:
  lines=q.split("\n")
  q_obj=[]
  e=len(lines)
  for l in lines:
    if(l=="" or l==" "):
      e=lines.index(l)
      break
  lines=lines[:e]
  # print(lines)

  for i in range(1,len(lines),2):
    # print(i)
    q_obj.append({"question":lines[i],"answer":lines[i+1]})
  questions_processed.append(q_obj)
questions_processed[:5]

[[{'question': "Main Question: I'm interviewing Sneha. Hi, Sneha.",
   'answer': "Answer: Hello, Nidi. How are you? I'm good."},
  {'question': 'Follow-up Question: Very good. Can you tell me about your background including your age group, profession, and education level?',
   'answer': "Answer: I'm 21, and I'm pursuing pursuing BTEC CSA."},
  {'question': 'Follow-up Question: How would you describe your personality type?',
   'answer': 'Answer: My personality type? Someone who loves love. Someone who loves human. Someone who loves to take a walk around. Okay.'},
  {'question': 'Follow-up Question: How often do you experience stress or anxiety, and what methods do you use to cope with it?',
   'answer': "Answer: Oh, I experience anxiety a lot lately, and it's been hard to cope with it. But I'm trying to, you know, get my way out of it. So I'm trying new things. I try to wake up morning in the morning, and I go outside, take a walk. I just speak with myself, listen to the song that give

In [7]:
device = 0 if torch.cuda.is_available() else -1

emotion_classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=1, device=device)

print(device)


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


0


In [8]:
@tool("emotional_analysis", return_direct=True)
def analyze_emotion(input: str) -> str:
    """
    Analyze the input text and return the detected emotion using a pre-trained model.

    Args:
        input (str): The input text to analyze.

    Returns:
        str: A string indicating the detected emotion.
    """
    predictions = emotion_classifier(input)

    if isinstance(predictions, list) and len(predictions) > 0:
        if isinstance(predictions[0], list) and isinstance(predictions[0][0], dict):
            emotion = predictions[0][0]['label']
            return emotion
        else:
            return "Emotion detection failed: Unexpected prediction format"
    else:
        return "Emotion detection failed: No predictions returned"



In [9]:
# Define prompts that align with different emotions/nature tags
nature_prompts = {"anger": "Answer the following question with a tone of anger: {input}. \nRespond in a way that expresses frustration or intense irritation.",
    "disgust": "Answer the following question with a tone of disgust: {input}. \nRespond in a way that conveys repulsion or strong aversion.",
    "fear": "Answer the following question with a tone of fear: {input}. \nRespond in a way that shows anxiety or concern.",
    "joy": "Answer the following question with a tone of joy: {input}. \nRespond in a way that reflects happiness or excitement.",
    "neutral": "Answer the following question with a neutral tone: {input}. \nRespond in a way that is calm, objective, and balanced.",
    "sadness": "Answer the following question with a tone of sadness: {input}. \nRespond in a way that expresses sorrow or melancholy.",
    "surprise": "Answer the following question with a tone of surprise: {input}. \nRespond in a way that conveys astonishment or amazement.",
}


In [10]:
agents = {}
for nature, prompt in nature_prompts.items():
    prompt_template = PromptTemplate.from_template(prompt)
    agents[nature] = prompt_template
agents

{'anger': PromptTemplate(input_variables=['input'], template='Answer the following question with a tone of anger: {input}. \nRespond in a way that expresses frustration or intense irritation.'),
 'disgust': PromptTemplate(input_variables=['input'], template='Answer the following question with a tone of disgust: {input}. \nRespond in a way that conveys repulsion or strong aversion.'),
 'fear': PromptTemplate(input_variables=['input'], template='Answer the following question with a tone of fear: {input}. \nRespond in a way that shows anxiety or concern.'),
 'joy': PromptTemplate(input_variables=['input'], template='Answer the following question with a tone of joy: {input}. \nRespond in a way that reflects happiness or excitement.'),
 'neutral': PromptTemplate(input_variables=['input'], template='Answer the following question with a neutral tone: {input}. \nRespond in a way that is calm, objective, and balanced.'),
 'sadness': PromptTemplate(input_variables=['input'], template='Answer the

In [ ]:
from huggingface_hub import login

hf_token = "HF_TOKEN"

login(token = hf_token)

wb_token = "WB_TOKEN"

wandb.login(key=wb_token)
run = wandb.init(
    project='Fine-tune Llama 3 8B on Cognitive Dataset', 
    job_type="training", 
    anonymous="allow"
)

In [12]:
base_model = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"
new_model = "llama-3-8b-chat-cognitive"

In [13]:
torch_dtype = torch.float16
attn_implementation = "eager"

In [14]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation
)
model_base=model;

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [15]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)
model, tokenizer = setup_chat_format(model, tokenizer)

In [16]:
# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)
model = get_peft_model(model, peft_config)


In [17]:
def data_loader(raw_qns):
    for q_s in raw_qns:
        root_q=q_s[0]["question"].split(": ")[1]
        emo=analyze_emotion(root_q)
        history={}
        agent_prompt = agents.get(emo, agents["neutral"])
        query = f"You are {name} in an interview. Now, the interviewer has asked a question which is given below. Give direct answer without writing anything extra in start or end including headings etc: \n\n"+agent_prompt.format(input=root_q)+"\n\n Give complete answer in one paragraph only, Just write answer to question and nothing else."
        q_obj={}
        q_obj["question"]=query
        act_ans=q_s[0]["answer"].split(": ")[1]
        q_obj["answer"]=act_ans
        
        yield q_obj
        history[root_q]=act_ans
        for i in range(1,len(q_s)):
            question=q_s[i]["question"].split(": ")[1]
            answer=q_s[i]["answer"].split(": ")[1]
            history_str = "\n".join([f"{question} -> {answer}" for question, answer in history.items()])
            agent_prompt = agents.get(emo, agents["neutral"])
            query = agent_prompt.format(input=question)

            prompt = (
                f"You are {name} continuing a conversation in an interview. Give direct answer to the following question without writing anything extra includings headings etc.:\n{query}\n\nBelow is the history of the conversation done previously for context in the format \"question -> answer\" :\n\n"
                f"{history_str}\n\n"
                "Please provide direct answer in the specified tone based on the previous conversation. Complete answer in one paragraph only, Just write answer to question and nothing else."
            )
            q_obj={"question":prompt, "answer":answer}
            yield q_obj
            history[question]=answer

In [18]:

def chat_template(row):
    row_json = [{"role": "user", "content": row["question"]},
               {"role": "assistant", "content": row["answer"]}]
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row


In [19]:
ds=data_loader(questions_processed)
import pandas as pd
dataset=[]
for obj in ds:
    chat_template(obj)
    dataset.append(obj)
dataset=pd.DataFrame(dataset)

/tmp/ipykernel_24/2442004934.py:4: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use invoke instead.
  emo=analyze_emotion(root_q)


In [20]:
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

dataset = Dataset(pa.Table.from_pandas(dataset))
### convert to Huggingface dataset
# hg_dataset = Dataset(pa.Table.from_pandas(dataset))



In [21]:
dataset = dataset.train_test_split(test_size=0.1)

In [22]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'text'],
        num_rows: 47
    })
    test: Dataset({
        features: ['question', 'answer', 'text'],
        num_rows: 6
    })
})

In [23]:
training_arguments = TrainingArguments(
    output_dir=new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="wandb"
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [24]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    peft_config=peft_config,
    max_seq_length=512,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will over

Map:   0%|          | 0/47 [00:00<?, ? examples/s]

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

In [25]:
from numba import cuda
# device = cuda.get_current_device()
# device.reset()



In [26]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss
5,5.261600,4.373101
10,3.931400,3.074186
15,2.628600,2.217543
20,1.309800,1.645236


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


TrainOutput(global_step=23, training_loss=3.1754237517066626, metrics={'train_runtime': 142.8187, 'train_samples_per_second': 0.329, 'train_steps_per_second': 0.161, 'total_flos': 853098852188160.0, 'train_loss': 3.1754237517066626, 'epoch': 0.9787234042553191})

In [27]:
trainer.save_model("tuned_model")

/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


In [28]:
!zip -r file.zip /kaggle/working/tuned_model

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  adding: kaggle/working/tuned_model/ (stored 0%)
  adding: kaggle/working/tuned_model/tokenizer.json (deflated 74%)
  adding: kaggle/working/tuned_model/training_args.bin (deflated 51%)
  adding: kaggle/working/tuned_model/adapter_config.json (deflated 53%)
  adding: kaggle/working/tuned_model/README.md (deflated 66%)
  adding: kaggle/working/tuned_model/special_tokens_map.json (deflated 66%)
  adding: kaggle/working/tuned_model/adapter_model.safetensors (deflated 22%)
  adding: kaggle/working/tuned_model/tokenizer_config.json (deflated 96%)


In [29]:
from IPython.display import FileLink
FileLink(r'file.zip')

/kaggle/working/file.zip

In [30]:

prompts = dataset['test']['question'][1]

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, 
                                       add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt', padding=True, 
                   truncation=True).to(model.device)

outputs = model.generate(**inputs, max_length=300,  # Adjust the max_length as needed
    num_return_sequences=1,  # Generate a single sequence
            )

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[1])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [31]:
def prepare_prompt(user_question):
    emo=analyze_emotion(user_question)
    agent_prompt = agents.get(emo, agents["neutral"])
#     print(user_question)
#     print("----------------")
    query = f"You are {name} in an interview. Now, the interviewer has asked a question which is given below. Give direct answer without writing anything extra in start or end including headings etc: \n\n"+agent_prompt.format(input=user_question)+"\n\n Give complete answer in one paragraph only, Just write answer to question and nothing else."
    return query

def generate_response(user_question):
    
    prompt=prepare_prompt(user_question)
#     print(prompt)
#     print("------------------")

    messages = [
        {
            "role": "user",
            "content": prompt
        }
    ]

    prompt = tokenizer.apply_chat_template(messages, tokenize=False, 
                                       add_generation_prompt=True)

    inputs = tokenizer(prompt, return_tensors='pt', padding=True, 
                   truncation=True).to(model.device)

    outputs = model.generate(**inputs, max_length=250,num_return_sequences=1)

    text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return (text.split("assistant")[1])
#     print(text)
# 

In [32]:
test_question ="How do you handle depression?"
print(generate_response(test_question))

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



I'm okay. I'm okay. I'm okay. I'm okay. I'm okay. I'm okay. I'm okay. I'm okay. I'm okay. I'm okay. I'm okay. I'm okay. I'm okay. I'm okay. I'm okay. I'm okay. I'm okay. I'm okay. I'm okay. I'm okay. I'm okay. I'm okay. I'm okay. I'm okay. I'm okay. I'm okay. I'm okay. I'm okay. I'm okay. I'm okay. I'm okay. I'm okay. I'm okay. I'm okay. I'm okay. I'm okay. I'm okay. I'm okay. I'm okay. I'm okay


In [33]:
dataset['test']['question'][1]

'You are Sneha continuing a conversation in an interview. Give direct answer to the following question without writing anything extra includings headings etc.:\nAnswer the following question with a neutral tone: Can you share an experience where greed negatively impacted your relationships or decisions? Yes.. \nRespond in a way that is calm, objective, and balanced.\n\nBelow is the history of the conversation done previously for context in the format "question -> answer" :\n\nit for you to maintain social connections? And how do you nurture these relationships? -> Social connections as in clubswala thing or like Both personal and professional. Okay. It\'s equally important for me to have social connections and to have, like, a workwala thing because it\'s like, okay. You\'re working 247. You\'re doing things. But if you don\'t have people around you, be it strangers or, like, if you don\'t feel those emotions, what\'s the point? What are you even looking forward to? If you get yourself